# Transformando dados passados no formato JSON

In [1]:
import json
from pathlib import Path
from datetime import datetime

## Funções

### Gerais

In [2]:
def load_data(path):
    """
    Load JSON from file at `path` (str).
    """
    #aux.log('Will load data from {:}'.format(path))
    return json.loads(Path(path).read_text(encoding="utf-8"))

In [3]:
def save_data(data, path):
    """
    Save `data` (dict) to `path` (str) if edit controls
    are enabled.
    """
    # Save data:
    with open(path, 'w') as f:
        json.dump(data, f, indent=1, ensure_ascii=False)

In [4]:
def ucop(data, func, update_date=True, **kwargs):
    """
    Apply function `func` to every usecase in `data`, in
    place.

    Parameters
    ----------
    data : dict
        JSON-like data with two primary keys: 'metadata' 
        and 'data'. The former must hold a dict with a 
        key 'ĺast_update'. The latter must be a list of 
        usecases on which to apply `func`.
    func : callable
        Function to apply to each usecase in `data['data']`.
        Its first argument must the the usecase.
    kwargs : dict
        Extra parameters to pass to `func`.
    update_date : bool
        Whether to update 'last_update' in the metadata.
    """
    
    # Loop over usecases to apply function:
    usecases = data['data']
    for uc in usecases:
        func(uc, **kwargs)

    # Update last_update:
    if update_date == True:
        data['metadata']['last_update'] = datetime.today().strftime('%Y-%m-%d')

In [5]:
def print_property(uc, key):
    print(uc[key])

### Transformações

In [6]:
def split_status(uc):
    """
    Split the 'status' property of a usecase `uc` (dict)
    in boolean 'status_published' and 'status_review'.
    """
    s = uc['status']
    if s == 'Oculto':
        uc['status_published'] = False
        uc['status_review'] = False
    elif s == 'Em revisão':
        uc['status_published'] = False
        uc['status_review'] = True
    elif s == 'Em validação':
        uc['status_published'] = True
        uc['status_review'] = True
    elif s == 'Publicado':
        uc['status_published'] = True
        uc['status_review'] = False
    else:
        raise Exception('Unknown status "{:}".'.format(s))

    # Remove status key from usecase:
    uc.pop('status')

## Transformações

### Split status

In [7]:
data = load_data('../analises/usecases_temp.json')

In [8]:
ucop(data, split_status)

In [10]:
#save_data(data, '../codigo/data/usecases_temp.json')